In [1]:
import sys
sys.path.append('../')

# Terms, Formulas and Interpretations

Now we have all the elements to formally define ```Tarski``` languages:

**Definition** (Many-Sorted First-Order Language). A _many-sorted_ _first-order_ language ${\cal L}$ is made up of:
 - A non-empty set $T$ of _sorts_
 - An _infinite number_ of _variables_ $x_{1}^{\tau}, x_{2}^{\tau}, \ldots$ for each short $\tau \in T$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n+1}$ of sorts, a (possibly empty) set of _function_ symbols, each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n+1})$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n}$ of sorts, a (possibly empty) set of _relation_ symbols (predicates), each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n})$

Continuing with our ```Blocks World``` themed example

In [2]:
import tarski
from tarski.symbols import *

# 1. Create language used to describe world states and transitions
bw = tarski.language()

# 2. Define sorts
place = bw.sort('place')
block = bw.sort('block', [place])

# 3. Define functions
loc = bw.function( 'loc', block, place )

# 4. Define predicates
clear = bw.predicate( 'clear', block)

_Constants_ are 0-arity functions, whose sort $\tau$ is a set with one single element. Hence, we handle them separatedly, as we specialise their representation

In [3]:
# 5. Define constants
b1, b2, b3, b4 = bw.const(('b_{}'.format(k) for k in (1,2,3,4)), block)
table = bw.const('table', place)

## (First-Order) Terms

Combinations of variables, functions and constants are called _terms_, and the rules for constructing them are given inductively:

**Definition** (First-Order Terms). A term $t$ can be:

 - Any variable $x^{\tau}$ of the language can be a term $t$ with type $\tau$
 - Any constant symbol of the language with type $\tau$ is a term with the same type
 - If $t_1, \ldots, t_n$ are terms with respective types $\tau_1, \ldots, \tau_n$ and $f$ is a _function_ symbol with type $(\tau_1, \ldots, \tau_n, \tau{n+1})$ then $f(t_1,\ldots,t_n)$ is a term with type $\tau_{n+1}$.

Terms are implemented as Python objects. Every constant symbol is an instance of ```Term```

In [4]:
from tarski import Term

isinstance(b1,Term)

True

Function symbols allow to nest terms, thus 

In [5]:
t1 = loc(b1)
isinstance(t1,Term)

True

In [6]:
x = bw.var('x', block)
t2 = loc(x)
isinstance(t2,Term)

True

In [7]:
t3 = loc( loc(x) )
isinstance(t3,Term)

True

are all terms. ```Tarski``` textual representation of variables is a bit different

In [8]:
print('{}, type: {}'.format(t1, t1.type))
print('{}, type: {}'.format(t2, t2.type))
print('{}, type: {}'.format(t3, t3.type))

loc(b_1), type: Sort(place)
loc(?x), type: Sort(place)
loc(loc(?x)), type: Sort(place)


in order to make distinct variables from constants, the former are printed with the prefix ```?```. 

## Formulas

Formulas (statements that can be either ```True``` or ```False```) are defined also inductively as follows:

**Definition** (First-Order Formulas).

 - If $t_1$ and $t_2$ are two terms with the same type, then $t_1 = t_2$ is an _atomic formula_.
 
 - If $t_1,\ldots,t_n$ are terms with respective types $\tau_1,\ldots,\tau_n$, and $R$ is a relation symbol with type $(\tau_1,\ldots,\tau_n)$, then $R(t_1,\ldots,t_n)$ is an atomic formula too.
 
 - If $\phi_1$ and $\phi_2$ are formulas then $\neg \phi_1$, $\phi_1 \lor \phi_2$ and $\phi_1 \land \phi_2$ are also formulas.
 
 - If $\phi$ is a formula, then $\exists_t x^{\tau}\, \phi$ and $\forall_t x^{\tau}\, \phi$ are also formulas.

Quantification happens over a certain sort, i.e. for each sort $\tau$ $\in$ $T$ there are universal and existential quantifier symbols $\forall_{\tau}$ and $\exists_{\tau}$, which may be applied to variables of the same sort.

Formulas without existential ($\exists$) or universal ($\forall$) quantifiers are called _quantifier free_.

### Examples

#### Relational (atomic) formulas

$t_1 = t_3$ - terms $t_1$ and $t_3$ are equal

In [9]:
t1==t3

We introduce the function $width(b)$ for blocks $b$, this will allow us to specify Hanoi Towers like tasks

In [10]:
width = bw.function('width', place, bw.Real)

We need a new variable so we can make general statements about more than one block

In [11]:
y = bw.var('y', block)

Now we can state properties of states like _for every block x, x cannot be wider than the place below_

$$
\forall x,y\, loc(x) = y \supset width(x) < width(y)
$$

which can be written as

In [12]:
forall( x, y, implies( loc(x) == y, width(x) < width(y) ) )

or alternatively

In [13]:
forall( x, y, (loc(x) == y) > (width(x) < width(y)) )

We can write the formula

$$
loc(b1) \neq loc(b2) \land loc(b1) \neq loc(b3)
$$

like

In [14]:
land( loc(b1) != loc(b2), loc(b1) != loc(b3))

or

In [15]:
(loc(b1) != loc(b2)) & (loc(b1) != loc(b3))

Another state invariant like 

$$
loc(b1) = b2 \lor loc(b1) = b3
$$

can be written as

In [16]:
lor( loc(b1) == b2, loc(b1) == b3 )

or

In [17]:
(loc(b1)==b2) | (loc(b1)==b3)

Finally, the formula 

$$
loc(b1) = b2 \supset \neg clear(b2)
$$

can be written as 

In [18]:
phi=implies( loc(b1) == b2, neg(clear(b2)))
phi

or, alternatively the ```~``` unary operator can be used instead of ```neg(...)```

In [19]:
phi = implies( loc(b1) == b2, ~clear(b2))
phi

and for those who find ```implies(...)``` to be unwanted clutter, the operator ```>``` can be used to bind together two formulae

In [20]:
phi = (loc(b1) == b2) > ~clear(b2)
phi

Note how the implication has been automatically transformed into a disjunction using the equivalence

$$
p \supset q \equiv \neg p \lor q
$$

In [21]:
str(phi)

'or(neg(b_2 = loc(b_1)),neg(clear(b_2)))'

In [22]:
tau = loc(b1) == b2
print(tau)

b_2 = loc(b_1)


In [23]:
tau = width(b1) > width(b2)
print(tau)

width(b_1) > width(b_2)


## Interpretations

The semantics of a first--order language are formalised as the notion of _interpretation_, which is also typically referred in texts on computational logic as _model_ or _structure_. Informally, the interpretation of a first-order language is a "table" that allows to:

 1. Map a first--order _term_ to a constant symbol

### Built-in Function Symbols

A number of functions are already defined for the built-in sorts ```Real```, ```Integer``` and ```Natural```, in order to facilitate the definition of terms that model arithmetic operators, algebraic and transcendental functions

| Name | Syntax | Notes | Name | Syntax | Notes |
|------|:--------|:-------------------|:----------|:-----|:------|
| Addition | `x + y` |          | Matrix Multiplication | `x @ y` |  |  
| Subtraction | `x - y` |       |  | |  | 
| Multiplication | `x * y` |    | | |  |
| Division | `x / y` |          | | ||
| Modulo | `x % y` |            | | |  |
| Power | `x ** y` | |           | | |  | 


#### Notes

#### Examples

In [24]:
a = bw.var('a', bw.Real)
b = bw.var('b', bw.Real)
a + b

LanguageError: FOL.resolve_function_symbol_2(): function symbol '+' is not defined for domain (Sort(Real),Sort(Real))

In [ ]:
a - b

In [ ]:
a ** bw.const(2,'Integer')